# Tools  🔨🪚🔩⚙️⚒️🧲🧰🔧🪛🔩⚙️🔦🧭🔗⛓️🧮
Tools allow agents to 'Act' in the real world. 
Careful descriptions can help your agent discover how to use your tools.

LangChain supports many tool formats and tool sets. Here we will cover some common cases, but check the [docs](https://docs.langchain.com/oss/python/langchain/tools) for more information.

## Setup

Load and/or check for needed environmental variables

In [4]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=<not set>
LANGSMITH_API_KEY=****4abe
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ad-6


## Calculator example

In this example, the docstring and inferred arguments and argument types are used by the LLM to detetermine when and how to call the tool.

In [5]:
from typing import Literal

from langchain.tools import tool


@tool
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two real numbers."""
    print("🧮 Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model="ollama:gpt-oss:20b",
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

This invokes your calculator tool.

In [7]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is 3.1125 * 4.1234"}]}
)
print(result["messages"][-1].content)

🧮 Invoking calculator tool
\(3.1125 \times 4.1234 \;=\; 12.8340825\)


We can check the **metadata** in [LangSmith Observability](https://smith.langchain.com/public/b77bde6c-f0ad-4256-bfab-7d514ece3405/r) to see this.

The tool description can have a big impact. 
This may not  invoke your calculator tool because the inputs are integers.  (results vary from run to run)

In [8]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

🧮 Invoking calculator tool
3 × 4 = 12.


This often does not invoke the tool though the input are real numbers. (results vary from run to run)

In [9]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

🧮 Invoking calculator tool
\(3.0 \times 4.0 = 12.0\)


## Adding a more detailed description
While a basic description is often sufficient, LangChain has support for enhanced descriptions. The example below uses one method: Google Style argument descriptions. Used with `parse_docstring=True`, this will parse and pass the arg descriptions to the model. You can rename the tool and change its description. This can be effective when you are sharing a standard tool but would like agent-specific instructions.

In [10]:
from typing import Literal

from langchain.tools import tool


@tool(
    "calculator",
    parse_docstring=True,
    description=(
        "Perform basic arithmetic operations on two real numbers."
        "Use this whenever you have operations on any numbers, even if they are integers."
    ),
)
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two real numbers.

    Args:
        a (float): The first number.
        b (float): The second number.
        operation (Literal["add", "subtract", "multiply", "divide"]):
            The arithmetic operation to perform.

            - `"add"`: Returns the sum of `a` and `b`.
            - `"subtract"`: Returns the result of `a - b`.
            - `"multiply"`: Returns the product of `a` and `b`.
            - `"divide"`: Returns the result of `a / b`. Raises an error if `b` is zero.

    Returns:
        float: The numerical result of the specified operation.

    Raises:
        ValueError: If an invalid operation is provided or division by zero is attempted.
    """
    print("🧮  Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [11]:
from langchain.agents import create_agent

agent = create_agent(
    model="ollama:gpt-oss:20b",
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

In [12]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

🧮  Invoking calculator tool
12.0


Let's check our [LangSmith Observability trace](https://smith.langchain.com/public/7d65902c-bd3c-4fc6-bbd3-7c1d66566fda/r) to see the tool description.

In [13]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

🧮  Invoking calculator tool
3 × 4 = 12.


## Try your own.
Create a tool of your own and try it here!

In [14]:
import os
import platform
import re
import subprocess
from datetime import datetime
from typing import Any, Dict

from langchain.tools import tool


@tool(
    "system_resource_snapshot",
    parse_docstring=True,
)
def system_resource_snapshot() -> Dict[str, Any]:
    """Capture current CPU load averages and memory usage on macOS or Unix systems.

    Returns:
        Dict[str, Any]: A dictionary with top-level ``cpu`` and ``memory`` keys. The
            ``cpu`` entry contains the logical core count, recent load averages, and
            an instantaneous usage breakdown (when ``top`` output is available).
            The ``memory`` entry reports total, available, and used bytes using
            ``vm_stat`` on macOS or ``/proc/meminfo`` on Linux. Both sections also
            include raw values from the underlying OS utilities.
    """

    def _load_average() -> Dict[str, float]:
        try:
            load1, load5, load15 = os.getloadavg()
            return {"1m": load1, "5m": load5, "15m": load15}
        except OSError:
            return {}

    def _cpu_usage() -> Dict[str, Any]:
        system = platform.system().lower()
        if system == "darwin":
            cmd = ["top", "-l", "1", "-n", "0"]
            line_prefix = "CPU usage"
        else:
            cmd = ["top", "-b", "-n", "1"]
            line_prefix = "%Cpu(s):" if system == "linux" else "Cpu(s):"

        try:
            output = subprocess.check_output(cmd, stderr=subprocess.STDOUT, text=True)
        except (subprocess.CalledProcessError, FileNotFoundError):
            return {}

        for line in output.splitlines():
            if line.strip().startswith(line_prefix):
                matches = re.findall(r"([0-9]+\.?[0-9]*)%\s*([a-zA-Z]+)", line)
                if not matches:
                    continue
                usage = {}
                for value, label in matches:
                    usage[label.lower()] = float(value)
                usage["raw_line"] = line.strip()
                return usage
        return {}

    def _memory_stats() -> Dict[str, Any]:
        system = platform.system().lower()
        if system == "darwin":
            return _memory_stats_darwin()
        if system == "linux":
            return _memory_stats_linux()
        return {"error": f"Unsupported platform for detailed memory stats: {system}"}

    def _memory_stats_darwin() -> Dict[str, Any]:
        try:
            total_bytes = int(subprocess.check_output(["sysctl", "-n", "hw.memsize"]).strip())
        except (subprocess.CalledProcessError, FileNotFoundError, ValueError):
            total_bytes = None

        try:
            vm_output = subprocess.check_output(["vm_stat"], text=True)
        except (subprocess.CalledProcessError, FileNotFoundError):
            return {"total_bytes": total_bytes}

        page_size_match = re.search(r"page size of (\d+) bytes", vm_output)
        page_size = int(page_size_match.group(1)) if page_size_match else 4096

        stats: Dict[str, int] = {}
        for line in vm_output.splitlines():
            if ":" not in line:
                continue
            key, value = line.split(":", 1)
            try:
                stats[key.strip()] = int(value.strip().strip('.'))
            except ValueError:
                continue

        free_pages = stats.get("Pages free", 0) + stats.get("Pages speculative", 0)
        inactive_pages = stats.get("Pages inactive", 0)
        available_bytes = (free_pages + inactive_pages) * page_size
        used_bytes = (total_bytes - available_bytes) if (total_bytes is not None and available_bytes is not None) else None

        return {
            "total_bytes": total_bytes,
            "available_bytes": available_bytes,
            "used_bytes": used_bytes,
            "page_size": page_size,
            "raw": {
                "pages_free": stats.get("Pages free"),
                "pages_inactive": stats.get("Pages inactive"),
                "pages_speculative": stats.get("Pages speculative"),
                "pages_active": stats.get("Pages active"),
                "pages_wired": stats.get("Pages wired down"),
            },
        }

    def _memory_stats_linux() -> Dict[str, Any]:
        try:
            with open("/proc/meminfo", "r", encoding="utf-8") as handle:
                lines = handle.readlines()
        except FileNotFoundError:
            return {"error": "Unable to read /proc/meminfo"}

        meminfo: Dict[str, int] = {}
        for line in lines:
            if ':' not in line:
                continue
            key, value = line.split(':', 1)
            parts = value.strip().split()
            if not parts:
                continue
            try:
                amount = int(parts[0])
            except ValueError:
                continue
            multiplier = 1024 if len(parts) > 1 and parts[1].lower() == "kb" else 1
            meminfo[key] = amount * multiplier

        total = meminfo.get("MemTotal")
        available = meminfo.get("MemAvailable")
        if available is None:
            available = (
                meminfo.get("MemFree", 0)
                + meminfo.get("Buffers", 0)
                + meminfo.get("Cached", 0)
                + meminfo.get("SReclaimable", 0)
                - meminfo.get("Shmem", 0)
            )
        used = (total - available) if (total is not None and available is not None) else None

        return {
            "total_bytes": total,
            "available_bytes": available,
            "used_bytes": used,
            "raw": {k: meminfo.get(k) for k in ("MemFree", "Buffers", "Cached", "SReclaimable", "Shmem")},
        }

    snapshot = {
        "timestamp": datetime.utcnow().isoformat() + "Z",
        "platform": platform.platform(),
        "cpu": {
            "logical_cores": os.cpu_count(),
            "load_average": _load_average(),
        },
        "memory": _memory_stats(),
    }

    cpu_usage = _cpu_usage()
    if cpu_usage:
        snapshot["cpu"]["usage_percent"] = cpu_usage

    return snapshot


In [18]:
from langchain.agents import create_agent

agent = create_agent(
    model="ollama:gpt-oss:20b",
    tools=[system_resource_snapshot],
    system_prompt="You are system resource monitoring assistant. Provide detailed system resource snapshots when requested. Present the information in a clear and structured manner. Using mermaid syntax for visualization data analysis.",
)


In [19]:
result = agent.invoke({"messages": [{"role": "user", "content": "Take a system resource snapshot"}]})
print(result["messages"][-1].content)

/var/folders/0t/60hzjwl11r9blfgm7hphh65c0000gn/T/ipykernel_85537/2332795036.py:153: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat() + "Z",


**System Resource Snapshot – 2025‑10‑28 10:40:40 UTC**

| Metric | Value |
|--------|-------|
| **Platform** | macOS 26.0.1 (arm64‑mac) |
| **CPU** | 10 logical cores |
| **CPU Load Averages** | 1 min: **4.314** | 5 min: **5.038** | 15 min: **4.926** |
| **CPU Usage** | User : **8.53 %** | System : **13.46 %** | Idle : **77.99 %** |
| **Memory** | Total : **34 GB** (34 359 738 368 bytes) |
| | Available : **5.94 GB** (5 943 934 976 bytes) |
| | Used : **28.42 GB** (28 415 803 392 bytes) |
| | Page size : 16 KB |
| | Pages free : **5 099** |
| | Pages inactive : **357 388** |
| | Pages speculative : **302** |
| | Pages active : **357 992** |
| | Pages wired : **1 090 231** |

---

### Visual Overview (Mermaid)

```mermaid
%% CPU load & usage
pie title CPU Load (1‑5‑15 min)
  "1 min" : 4.314
  "5 min" : 5.038
  "15 min": 4.926

%% CPU usage distribution
pie title CPU Usage (Current)
  "User"   : 8.53
  "System" : 13.46
  "Idle"   : 77.99

%% Memory usage
pie title Memory Allocation
  "Us